<a href="https://colab.research.google.com/github/jonghhhh/lecture_colabs/blob/main/%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EB%84%A4%EC%9D%B4%EB%B2%84%EB%89%B4%EC%8A%A4%EB%B3%B8%EB%AC%B8_110924.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 네이버 뉴스 본문 수집

## 패키지 불러오기: urllib, BeautifulSoup, pandas

---



In [ ]:
from urllib.request import urlopen   # url에 접속
from bs4 import BeautifulSoup   # html을 구조적으로 나누고(parsing), 원하는 부분을 추출하는(select) 기능

import pandas as pd             # 수집된 내용을 dataframe 방식으로 저장
import pprint                   # 정리해서 보여주기

## 함수 만들기: 네이버 뉴스 url로 본문 수집

In [ ]:
import requests
from bs4 import BeautifulSoup

more_remove={"KBS": "■ 제보하기▷ 전화 : 02-781-1234, 4444▷ 이메일 : kbs1234@kbs.co.kr▷ 카카오톡 : 'KBS제보' 검색, 채널 추가▷ 네이버, 유튜브에서 KBS뉴스를 구독해주세요!",
"MBC": "MBC 뉴스는 24시간 여러분의 제보를 기다립니다. ▷ 전화 02-784-4000▷ 이메일 mbcjebo@mbc.co.kr▷ 카카오톡 @mbc제보",
"YTN" : "※ '당신의 제보가 뉴스가 됩니다'[카카오톡] YTN 검색해 채널 추가[전화] 02-398-8585[메일] social@ytn.co.kr"}

def text_collect(url):  # 네이버뉴스 url = 'https://n.news.naver.com/article/214/0001327361?type=main'
    response = requests.get(url)
    time.sleep(0.3)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        writer=soup.select('span.byline_s')[0].text
    except:
        writer = ''
    try:
        txt=soup.select('#dic_area')[0].text
        for rem in [remove.text for remove in soup.select('em.img_desc')]+[writer]:
            txt=txt.replace(rem, '')
        txt=txt.replace('\t', '').strip()
        if 'article/056' in url: txt=txt.replace(more_remove['KBS'], '').strip()
        elif 'article/214' in url: txt=txt.replace(more_remove['MBC'], '').strip()
        elif 'article/052' in url: txt=txt.replace(more_remove['YTN'], '').strip()
        else: pass
    except:
        txt=''
    try:
        upload_date=soup.select('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')[0].text
    except:
        upload_date=''
    try:
        section=soup.select('em.media_end_categorize_item')[0].text
    except:
        section = ''
    return [url, upload_date, section, writer, txt]    # 입력날짜시간, 뉴스섹션, 작성자, 본문 출력

## 실습과 저장

In [ ]:
# 실습(1)
url='https://n.news.naver.com/mnews/article/008/0005016916?sid=101'
text_collect(url)

In [ ]:
# 실습(2): 복수의 기사 본문 수집
result_df=pd.read_excel('/content/drive/MyDrive/제주43/result_KBS.xlsx')
result_df.info()

from tqdm import tqdm
import time

texts=[]
for url in tqdm(list(result_df['url_naver'])[:]):     # 앞의 result_df에서 수집한 url 불러와 수집
    texts.append(text_collect(url))
    time.sleep(0.5)
result_text=pd.DataFrame(texts, columns=['url_naver', 'upload_date', 'section', 'writer', 'full_text'])  # 수집 결과를 dataframe으로 구성

result_all=pd.merge(result_df, result_text, on='url_naver', how='left')    # result_df와 result_text를 url 기준으로 결합

In [ ]:
# 저장

result_all.to_excel('/content/drive/MyDrive/제주43/result_KBS_본문.xlsx')  # 자신의 작업 폴더에 저장